In [2]:
# -*- coding: utf-8 -*-
"""vrp_dqn.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1iothquIlGtKKte5KIxO-YCKzXnZGjbK-
"""
# !pip install tsplib95

from tqdm import tqdm
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib import style
import tsplib95
import networkx as nx
import pickle

import time
import math
import re
import random
import copy

import tensorflow
if tensorflow.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tensorflow.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

from tensorflow.python.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from tensorflow.python.keras import Sequential
# from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.optimizers import Adam
from collections import deque

import io # to save the image

Default GPU Device: /device:GPU:0


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
# https://stackoverflow.com/questions/57301698/how-to-change-a-learning-rate-for-adam-in-tf2https://stackoverflow.com/questions/57301698/how-to-change-a-learning-rate-for-adam-in-tf2
class LearningRateReducerCb(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # old_lr = self.model.optimizer.lr.read_value()
        # new_lr = old_lr * 0.99
        new_lr = 0.1
        # print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

In [14]:
# 6. Define the movement restriction of the truck.
class Truck:
    def __init__(self, capacity, id, color):
        self.id = id
        self.color = color
        self.path = [] #this has the list of nodes it has visited
        self.max_truck_capacity = copy.deepcopy(capacity) #the max capacity
        self.capacity = copy.deepcopy(capacity)
        #self.visit_depo()
        self.prev_node = None
        self.node = 1 #starts from the depo

    def action(self, choice):
        # the number of choice of actions are the number of nodes-1
        # the choice to be taken depends on the demands - penalty based
        # the choice number is the same as the node number
        # it is not a choice if the demand is 0 - changing this to penalty
        #!! Want the system to learn instead
        # if self.capacity == 0:
        #     self.visit_depo()
        self.move(choice)

    def move(self, to_node_value):
        # node_list_copy = copy.deepcopy(node_list)
        # node_list_copy.remove(1)
        # select a random node to go to
        #if not to_node_value: #to_node_value is False by default
        #    to_node_value = random.choice(self.node_list)
        if to_node_value == 1:
            self.visit_depo()
        self.prev_node = self.node
        self.node = to_node_value
        self.path.append(to_node_value)
        # when invoked update the demand of the node
        # update the demand of the node

    def visit_depo(self):
        self.prev_node = self.node
        self.node = 1 #here it is 1
        self.capacity = copy.deepcopy(self.max_truck_capacity) #truck capacity reset
        self.path.append(1)
    
    #def path(self, node_value):
    #    self.path.append(node_value)

    # def get_node(self):
    #     return self.node

    
    # def get_capacity(self):
    #     return self.capacity


In [79]:
class VRPEnvironment:
    # environment related constants
    #https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.figure.html
    #observation_space = (640,480,3)
    # observation_space = (100,100,3)
    observation_space = (10,10,3)
    # penalty and rewards
    non_positive_capacity_penalty = 100
    zero_demand_penalty = 100 #truck goes to a zero demand node - except 1
    # ignore the failing task penalty as the episode will end only when...
    # all the demands are satisfied i.e. task will always be success...
    # The penalty of the achievement needs to go down.
    failing_task_penalty = 100 #trucks fail to complete the task
    completion_reward = 5000 #trucks complete the task
    hopping_incentive_penalty = 1000
    # visit_correct_node_reward = 100
    # exploration settings
    permitted_path_length = 20 #30
    epsilon = 1
    epsilon_decay = 0.999#changing this from 0.999
    min_epsilon = 0.001 #0.001
    no_of_episodes = 10_000 #30_000
    # from 0 to 0.5 difference is small so using 1 first
    truck_colors = {
        1:(0,0,1),
        2:(0,1,0),
        3:(1,0,0),
        4:(0,0.5,0.5),
        5:(0.5,0,0.5),
        6:(0.5,0.5,0),
        7:(0.5,0.5,0.5),
        8:(0.5,0.5,1),
        9:(0.5,1,0.5),
        10:(1,0.5,0.5)
    }
    return_images = True
    image_size = 100

    def __init__(self):
        # 1. Extract the tsplib95 file problem
        # self.problem = tsplib95.load_problem('/content/drive/My Drive/HW Assignments/Sem 2/ADBI/Vrp-All/A/A-n32-k5.vrp')
        # self.problem = tsplib95.load_problem('/content/drive/My Drive/HW Assignments/Sem 2/ADBI/Vrp-All/_singleTruck/A-n32-k5_2.vrp')
        self.problem = tsplib95.load_problem('Vrp-All/_singleTruck/A-n32-k5_3.vrp')        
        # 2. Create a networkx graph out of the problem. //will be plotting this
        self.nx_graph = self.problem.get_graph()
        self.edge_list = list(self.problem.get_edges()) #[(,)]
        self.node_positions = self.problem.node_coords #dict
        # the list of nodes
        self.node_list = list(self.problem.get_nodes())
        self.action_space = len(self.node_list) #the number of choices including staying put
        # the depot location
        self.depot_location = 1
        # assigning the default color of the nodes to be black
        # node_color_values = [(0,0,0) for i in range(len(node_list))]
        # reseting the environment when initialized
        self.reset_environment()
        
    def reset_environment(self):
        # creating the Trucks
        # 4. Extract the necessary data about the trucks. //no of trucks, depot_section, capacity
        self.node_demands = copy.deepcopy(self.problem.demands)
        truck_capacity = copy.deepcopy(self.problem.capacity)
        # trying hardcoding for now
        self.truck = Truck(truck_capacity, 1, self.truck_colors.get(3))

        self.episode_step = 0

        if self.return_images:
            observation = np.array(self.get_image())
        
        # resetting the environment reward value
        # self.reward = 0
        # there is no else case as we need always need the image for CNN
        return observation

    # def calculate_move_penalty(self, source_node, dest_node):
    #     return self.problem.wfunc(source_node, dest_node) #the weight of the edge

    # change the demand of the node when visited
    def change_demand(self, node):
        self.node_demands[node] = 0

    def step(self, action):
        self.episode_step += 1
        # moving the trucks for the action
        self.truck.action(action)
        #print("action="+str(action)+"\n++++++")
        self.truck.capacity -= self.node_demands.get(action)
        self.change_demand(action)
        # print(self.truck.capacity)
        # other truck actions

        if self.return_images:
            new_observation = np.array(self.get_image())
        # there is no else case as the return image is always true

        # assinging the rewards and penalties
        self.reward = 0
        # checking if the demands have been satisfied
        completed = False
        if sum(list(self.node_demands.values())) == 0:
            print("\n***Satisfied***")
            # print(self.node_demands)
            completed = True
            self.reward = self.completion_reward
        else:
            # rewards for other trucks
            # print("before"+str(self.reward))
            self.node_penalty(self.truck) #other penalties
            self.movement_penalty(self.truck) #edge weight
            # print("after"+str(self.reward))
            # penalties for other trucks
        done = False
        if self.reward == self.completion_reward or len(self.truck.path) >= self.permitted_path_length:
        # if self.reward == self.completion_reward:
            done = True
            if sum(list(self.node_demands.values())) > 0:
                self.reward -= self.failing_task_penalty

        return new_observation, self.reward, done, completed

    def node_penalty(self, truck):
        if self.node_demands[truck.node] == 0:
            if (truck.node == 1 and truck.capacity == truck.max_truck_capacity) or truck.node!=1:
                self.reward -= self.zero_demand_penalty

        if self.truck.capacity <= 0:
            self.reward -= self.non_positive_capacity_penalty
        # return self.reward

    def movement_penalty(self, truck):
        #print(self.truck.path)
        #print("-------")
        if truck.prev_node: #else it's 0
            source_node = truck.prev_node
            destination_node = truck.node
            if source_node == destination_node: #if truck stays at the same place
                self.reward -= self.hopping_incentive_penalty
            self.reward -= self.problem.wfunc(source_node, destination_node)
            # return self.problem.wfunc(source_node, destination_node)
        # else:
        #     source_node = truck.node
        #     destination_node = truck.node
        #     return 0
    # """
    def get_image(self):
        # the initiated rgb image of the given size. image_size = 100
        env = np.zeros((self.image_size, self.image_size, 3), dtype=np.uint8)
        for node in self.node_positions.keys():
            node_coods = self.node_positions.get(node)
            env[int(node_coods[0])][int(node_coods[1])] = (255,255,255)
        if self.truck.path: #if there are elements in the path
            for visited_node in set(self.truck.path):
                node_coods = self.node_positions.get(visited_node)
                #print(node_coods)
                #print(visited_node)
                #print(self.truck.path)
                #print(len(env))
                #print("--------")
                #try: #HAVING 0 HERE WAS CAUSING ERRORS! IGNORING 0 FOR NOW- STUPID
                env[int(node_coods[0])][int(node_coods[1])] = (255,0,0)
                #except:
                    #print(node_coods)
                    #print(visited_node)
                    #print(self.truck.path)
                    #print(len(env))
                    #print("--------")
        img = Image.fromarray(env, 'RGB')
        # trying to reduce to size to decrease the time taken
        img = img.resize((10,10))
        return img

    def render(self):
        img = self.get_image()
        img = img.resize((500,500))
        cv2.imshow("image", np.array(img))
        cv2.waitKey(1)
    """
    def get_image(self):
        fig,ax = plt.subplots()
        node_color_values = [(0,0,1) for i in range(len(self.node_list))]
        # ax.clear()
        if not self.truck.path: #if it is not empty
            for i in self.truck.path:
                node_color_values[i] = (1,0,0)
        image_io = io.BytesIO()
        #edge color is white to ignore it
        nx.draw(self.nx_graph, pos=self.node_positions, with_labels=False, node_color=node_color_values, node_size=20, edge_color=(1,1,1))
        fig.savefig(image_io, dpi=5)
        image = Image.open(image_io)
        image = image.resize((10,10))
        image.close() #keeping it open consumes a lot of memory
        return image

    def render(self):
        image = self.get_image()
        image = image.resize((1000,1000))
        cv2.imshow("image",np.array(image))
        cv2.waitKey(1)
    """

In [80]:
class DQNAgent:
    def __init__(self):
        # main model
        self.main_model = self.create_model()
        
        # target model
        self.target_model = self.create_model()
        self.target_model.set_weights(self.main_model.get_weights())

        # an array with last n steps for training
        self.replay_memory = deque(maxlen=considering_training_length)

        # used to know when to update target n/w with main n/w's weights
        self.target_update_counter = 0
        
    def create_model(self):
        model = Sequential()

        # model.add(Conv2D(256, (10,10), input_shape=environment.observation_space))
        model.add(Conv2D(256, (3,3), input_shape=environment.observation_space))
        model.add(Activation('relu'))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        # model.add(Conv2D(256,(10,10)))
        model.add(Conv2D(256,(3,3)))
        model.add(Activation('relu'))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        # converting the 3D features into the 1D feature
        model.add(Flatten())
        model.add(Dense(64))

        model.add(Dense(environment.action_space, activation='linear'))
        model.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])

        return model

    # adding the step data into the array to be considered later
    def update_replay_memory(self, step):
        self.replay_memory.append(step)
    
    def train(self, terminal_state, step):
        # start training only when we have a certain number of samples already saved
        if len(self.replay_memory)< min_replay_memory_size:
            return
        # get the minibatch of the samples from the replay table
        minibatch = random.sample(self.replay_memory, min_training_length)

        # get current states from minibatch, then query NN model for Q values
        current_states = np.array([transition[0] for transition in minibatch])/255
        current_qs_list = self.main_model.predict(current_states)

        # get future states from the minibatch, then query NN model for Q values
        new_current_states = np.array([transition[3] for transition in minibatch])/255
        future_qs_list = self.target_model.predict(new_current_states)

        X = []
        y = []
        
        # enumerating through the batches
        for index,(current_state, action, reward, new_current_state, done) in enumerate(minibatch):
            # if its not a terminal state, get new q from future states or else set to 0
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + discount*max_future_q
            else:
                new_q = reward

            # update the q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # append to the training data
            X.append(current_state)
            y.append(current_qs)
        
        # fit on all samples as one batch
        self.main_model.fit(np.array(X)/255, np.array(y), batch_size=min_training_length, verbose=0, shuffle=False, callbacks=[LearningRateReducerCb()])
        
        # updating the target n/w counter every episode
        if terminal_state:
            self.target_update_counter += 1
        
        # if the counter reaches the required value...
        # update the target n/w with weights of main n/w
        if self.target_update_counter > update_target_every:
            self.target_model.set_weights(self.main_model.get_weights())
            self.target_update_counter = 0
    
    # query the main n/w for q values given the current observation space
    def get_qs(self, state):
        return self.main_model.predict(np.array(state).reshape(-1, *state.shape)/255)[0]


In [81]:
# model related constants
discount = 0.99 #the discount applied to the DQN equation
# the environment keeps running till the demand of the nodes is satisfied
considering_training_length = 50_000 #the no of steps considered for training
min_training_length = 100 #the no of steps used for training
# episodes = 30_000
update_target_every = 5 #terminal states (end of episodes)
min_replay_memory_size = 1000 #min no of steps in a memory to start training

environment = VRPEnvironment()
agent = DQNAgent()

In [ ]:
show_preview = False #using this will make it slow. Don't care about this now.
aggregrate_stats_every = 50
start_decaying = True #making this True ignores the conditional decaying - Notes 2.a

x_axis = list(range(environment.no_of_episodes+1))
epsilon_list = []
rewards_list = []
path_length_list = []

epsilon_decaying_factor = 3000


# iterate over the episodes
for episode in tqdm(range(1, environment.no_of_episodes + 1), ascii=True, unit='episodes'):
    #print(episode)
    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1
    # Reset environment and get initial state
    current_state = environment.reset_environment()
    # Reset flag and start iterating until episode ends
    done = False
    while not done:
        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > environment.epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
            #print("action="+str(action)+"\n++++++")
        else:
            # Get random action
            action = np.random.randint(1, environment.action_space)

        if action != 0: #JUST IGNORING 0. MIGHT BE STUPID!
            new_state, reward, done, completed = environment.step(action)
        # Transform new continous state to new discrete state and count reward
        episode_reward += reward
        if show_preview and not episode % aggregrate_stats_every:
            env.render()
        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)
        current_state = new_state
        step += 1

        # Decay the Epsilon
        if completed:
            start_decaying = True
        
        if start_decaying:
            # environment.epsilon = 0.001
            if environment.epsilon > environment.min_epsilon:
                # environment.epsilon *= environment.epsilon_decay
                environment.epsilon = np.exp(-episode/epsilon_decaying_factor)
                environment.epsilon = max(environment.min_epsilon, environment.epsilon)
        # print(".",end="")
    epsilon_list.append(environment.epsilon)
    rewards_list.append(episode_reward)
    path_length_list.append(len(environment.truck.path))
    
    print("Path is:", end=" ")
    print(environment.truck.path, end=";")
    print("P and R: "+str(episode_reward), end=";")
    print("Capacity: "+str(environment.truck.capacity), end=";")
    print("Epsilon: "+str(environment.epsilon))











  0%|                                                                                                                                                                             | 0/10000 [00:00<?, ?episodes/s]









  0%|2                                                                                                                                                                  | 13/10000 [00:00<01:18, 127.75episodes/s]

Path is: [10, 2, 2, 10, 3, 4, 8, 5, 9, 8, 4, 2, 5, 5, 3, 1, 1, 3, 6, 8];P and R: -6025;Capacity: 76;Epsilon: 0.9996667222160499
Path is: [7, 8, 7, 2, 10, 7, 9, 5, 1, 1, 3, 5, 3, 7, 7, 6, 8, 8, 1, 1];P and R: -6578;Capacity: 100;Epsilon: 0.9993335555061811
Path is: [6, 9, 3, 5, 1, 1, 6, 4, 5, 1, 1, 6, 10, 1, 1, 10, 4, 7, 7, 5];P and R: -6505;Capacity: 88;Epsilon: 0.999000499833375
Path is: [1, 1, 4, 2, 1, 1, 9, 4, 5, 1, 1, 2, 3, 6, 1, 1, 7, 2, 4, 7];P and R: -6352;Capacity: 88;Epsilon: 0.9986675551606254
Path is: [6, 7, 1, 1, 1, 1, 8, 6, 7, 6, 3, 9, 9, 6, 10, 3, 9, 3, 1, 1];P and R: -6557;Capacity: 100;Epsilon: 0.9983347214509387

***Satisfied***
Path is: [10, 9, 6, 8, 4, 4, 10, 8, 2, 2, 7, 3, 9, 7, 4, 9, 7, 1, 1, 5];P and R: -503;Capacity: 86;Epsilon: 0.9980019986673331
Path is: [6, 7, 2, 4, 3, 6, 2, 4, 3, 4, 1, 1, 6, 4, 5, 7, 2, 2, 5, 9];P and R: -4829;Capacity: 85;Epsilon: 0.9976693867728395
Path is: [3, 5, 6, 7, 9, 10, 10, 6, 7, 6, 9, 5, 9, 9, 8, 3, 10, 6, 3, 1, 1];P and R: -6041;Ca











  0%|3                                                                                                                                                                   | 23/10000 [00:00<02:14, 74.26episodes/s]


***Satisfied***
Path is: [7, 7, 4, 8, 1, 1, 4, 9, 1, 1, 2, 9, 5, 2, 9, 4, 5, 8, 6];P and R: -300;Capacity: 54;Epsilon: 0.9926934899474861
Path is: [1, 1, 5, 10, 10, 9, 3, 10, 3, 1, 1, 8, 3, 5, 5, 2, 7, 5, 5, 8];P and R: -7592;Capacity: 62;Epsilon: 0.9923626472610146
Path is: [6, 4, 2, 10, 6, 2, 5, 8, 10, 10, 6, 8, 3, 9, 2, 9, 5, 3, 5, 5];P and R: -5259;Capacity: 14;Epsilon: 0.9920319148370607












  0%|4                                                                                                                                                                   | 29/10000 [00:00<02:44, 60.75episodes/s]

...

Path is: [8, 8, 8, 5, 6, 8, 8, 8, 8, 3, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8];P and R: -14525;Capacity: 32;Epsilon: 0.22961912216054609












 44%|#####################################################################7                                                                                        | 4415/10000 [4:05:31<5:33:48,  3.59s/episodes]

Path is: [8, 8, 8, 8, 8, 8, 8, 1, 1, 10, 10, 10, 10, 10, 10, 4, 3, 10, 10, 4];P and R: -15299;Capacity: 79;Epsilon: 0.22954259520835987












 44%|#####################################################################7                                                                                        | 4416/10000 [4:05:35<5:52:45,  3.79s/episodes]

Path is: [5, 5, 5, 5, 5, 5, 7, 5, 5, 1, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8];P and R: -17237;Capacity: 82;Epsilon: 0.22946609376090668


In [0]:
# # problem = tsplib95.load_problem('/content/drive/My Drive/HW Assignments/Sem 2/ADBI/Vrp-All/A/A-n32-k5.vrp')
# problem = tsplib95.load_problem('/content/drive/My Drive/HW Assignments/Sem 2/ADBI/Vrp-All/_singleTruck/A-n32-k5_2.vrp')
# nx_graph = problem.get_graph()
# edge_list = list(problem.get_edges()) #[(,)]N
# node_positions = problem.node_coords #dict
# node_demands = copy.deepcopy(problem.demands)
# truck_capacity = problem.capacity
# Try rewarding for each node too i.e incentivize hopping!!!!

In [0]:
# node_demands[7] = 0
# print(node_demands)
# print(problem.demands)
# print(sum(list(node_demands.values())))

In [ ]:
x = list(range(10000))
# print(x)
k = 1000
y = [float(np.exp(-i/k)) for i in x]
# print(y)
plt.plot(x,y)

In [ ]:
plt.plot(x_axis,epsilon_list,'r')
plt.plot(x_axis,rewards_list,'g')
plt.plot(x_axis,path_length_list,'b')

In [ ]:
plt.show()

In [14]:
problem = tsplib95.load_problem('Vrp-All/_singleTruck/A-n32-k5_3.vrp')
problem.wfunc(1,10)

11